In [1]:
import os
import yaml
import glob
import random
import numpy as np
import SimpleITK as sitk
from skimage.measure import regionprops

In [2]:
def ResampleXYZAxis(imImage, space=(1., 1., 1.), interp=sitk.sitkLinear):
    identity1 = sitk.Transform(3, sitk.sitkIdentity)
    sp1 = imImage.GetSpacing()
    sz1 = imImage.GetSize()

    sz2 = (int(round(sz1[0]*sp1[0]*1.0/space[0])), int(round(sz1[1]*sp1[1]*1.0/space[1])), int(round(sz1[2]*sp1[2]*1.0/space[2])))

    imRefImage = sitk.Image(sz2, imImage.GetPixelIDValue())
    imRefImage.SetSpacing(space)
    imRefImage.SetOrigin(imImage.GetOrigin())
    imRefImage.SetDirection([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0])
        # label.SetDirection([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0])
    # imRefImage.SetDirection(imImage.GetDirection())

    imOutImage = sitk.Resample(imImage, imRefImage, identity1, interp)

    return imOutImage

In [3]:
def ResampleLabelToRef(imLabel, imRef, interp=sitk.sitkLinear):
    identity1 = sitk.Transform(3, sitk.sitkIdentity)

    imRefImage = sitk.Image(imRef.GetSize(), imLabel.GetPixelIDValue())
    imRefImage.SetSpacing(imRef.GetSpacing())
    imRefImage.SetOrigin(imRef.GetOrigin())
    
    imRefImage.SetDirection([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0])

    # imRefImage.SetDirection(imRef.GetDirection())
        
    npLabel = sitk.GetArrayFromImage(imLabel)
    labels = np.unique(npLabel)
    resampled_nplabel_list = []
    for idx in labels:
        tmp_label = (npLabel == idx).astype(np.float32)
        tmp_imLabel = sitk.GetImageFromArray(tmp_label)
        tmp_imLabel.CopyInformation(imLabel)
        tmp_resampled_Label = sitk.Resample(tmp_imLabel, imRefImage, identity1, interp)
        resampled_nplabel_list.append(sitk.GetArrayFromImage(tmp_resampled_Label))
    
    one_hot_resampled_label = np.stack(resampled_nplabel_list, axis=0)
    resampled_label = np.argmax(one_hot_resampled_label, axis=0)
    outLabel = sitk.GetImageFromArray(resampled_label.astype(np.float32))
    outLabel.CopyInformation(imRef)

    return outLabel

In [4]:
def ResampleCRCMRImage(imImage, imLabel, img_save_path, mask_save_path, patient_name, count, target_spacing=(1., 1., 1.)):

    # assert imImage.GetSpacing() == imLabel.GetSpacing()
    assert round(imImage.GetSpacing()[0], 3) == round(imLabel.GetSpacing()[0], 3)
    assert round(imImage.GetSpacing()[1], 3) == round(imLabel.GetSpacing()[1], 3)
    assert round(imImage.GetSpacing()[2], 3) == round(imLabel.GetSpacing()[2], 3)
    assert imImage.GetSize() == imLabel.GetSize()


    spacing = imImage.GetSpacing()
    origin = imImage.GetOrigin()


    npimg = sitk.GetArrayFromImage(imImage)
    nplab = sitk.GetArrayFromImage(imLabel)
    z, y, x = npimg.shape

    # if not os.path.exists('%s'%(img_save_path)):
    os.makedirs('%s'%(mask_save_path), exist_ok=True)
    os.makedirs('%s'%(img_save_path), exist_ok=True)
    
    re_img_xy = ResampleXYZAxis(imImage, space=(target_spacing[0], target_spacing[1], spacing[2]), interp=sitk.sitkBSpline3)
    # re_lab_xy = ResampleLabelToRef(imLabel, re_img_xy, interp=sitk.sitkNearestNeighbor)
    re_lab_xy = ResampleLabelToRef(imLabel, re_img_xy, interp=sitk.sitkBSpline3)

    re_img_xyz = ResampleXYZAxis(re_img_xy, space=(target_spacing[0], target_spacing[1], target_spacing[2]), interp=sitk.sitkBSpline3)
    # re_lab_xyz = ResampleLabelToRef(re_lab_xy, re_img_xyz, interp=sitk.sitkNearestNeighbor)
    re_lab_xyz = ResampleLabelToRef(re_lab_xy, re_img_xyz, interp=sitk.sitkBSpline3)

    # sitk.sitkLinear


    # sitk.WriteImage(re_img_xyz, '%s/%s_%d.nii.gz'%(img_save_path, patient_name, count))
    # sitk.WriteImage(re_lab_xyz, '%s/%s_%d.nii.gz'%(mask_save_path, patient_name, count))
    sitk.WriteImage(re_img_xyz, '%s/%s.nii.gz'%(img_save_path, patient_name))
    sitk.WriteImage(re_lab_xyz, '%s/%s.nii.gz'%(mask_save_path, patient_name))

    print(patient_name, re_img_xyz.GetSpacing(), re_lab_xyz.GetSpacing())
    
    assert re_img_xyz.GetSpacing() == re_lab_xyz.GetSpacing()
    
    # print(round(spacing[0],4), round(spacing[1],4), round(spacing[2],4))
    # count += 1


In [5]:
# image_input = f'/home/workspace/AMP_mysef_3D_Cls/data/imagesTs/'
# mask_input  = f'/home/workspace/AMP_mysef_3D_Cls/data/labelsTs/'
target_spacing = (0.36, 0.36, 0.36)

# image_input = f'/home/workspace/research/AMP_mysef_3D_Cls/data/imagesTr/'
# mask_input  = f'/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTr/'
# image_out = '/home/workspace/research/AMP_mysef_3D_Cls/data/resampled_BS3_0525/imagesTr/'
# mask_out = f'/home/workspace/research/AMP_mysef_3D_Cls/data/resampled_BS3_0525/labelsTr/'


image_input = f'/home/workspace/research/AMP_mysef_3D_Cls/data/imagesTs/'
mask_input  = f'/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/'
image_out = '/home/workspace/research/AMP_mysef_3D_Cls/data/resampled_BS3_0525/imagesTs/'
mask_out = f'/home/workspace/research/AMP_mysef_3D_Cls/data/resampled_BS3_0525/labelsTs/'

os.makedirs(image_out +'/', exist_ok=True)
os.makedirs(mask_out +'/', exist_ok=True)


image_files = sorted(glob.glob(os.path.join(image_input + '*.nii.gz')))
mask_files  = sorted(glob.glob(os.path.join(mask_input + '*.nii.gz')))

# image_files, mask_files = image_files[:10], mask_files[:10]

count = 0
for data, seg in zip(image_files, mask_files):
    # print(image, label)
    tmp = data.split('/')[-1].split('.')[0]
    # print(tmp)
    patient_name = tmp

    data = sitk.ReadImage(data)
    seg = sitk.ReadImage(seg)

    # assert data.GetSpacing() == seg.GetSpacing()
    assert round(data.GetSpacing()[0], 3) == round(seg.GetSpacing()[0], 3)
    assert round(data.GetSpacing()[1], 3) == round(seg.GetSpacing()[1], 3)
    assert round(data.GetSpacing()[2], 3) == round(seg.GetSpacing()[2], 3)

    assert data.GetSize() == seg.GetSize()
    assert data.GetOrigin() == seg.GetOrigin()
    # assert data.GetDirection() == seg.GetDirection()

    # print(data.GetSpacing())

    # spacing = data.GetSpacing()

    # print(round(spacing[0],4), round(spacing[1],4), round(spacing[2],4))

    # ResampleCRCMRImage(data, seg, image_out, mask_out, patient_name, count, (round(spacing[1],4), round(spacing[1],4), round(spacing[1],4)))

    ResampleCRCMRImage(data, 
                       seg, 
                       image_out, 
                       mask_out, 
                       patient_name, 
                       count,
                       target_spacing 
                    #    (round(spacing[1],4), round(spacing[1],4), round(spacing[1],4))
                       )

    count += 1
    # print(name, 'done')

print(count)



crc_266 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_267 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_268 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_269 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_270 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_271 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_272 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_273 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_274 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_275 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_276 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_277 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_278 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_279 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_280 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_281 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_282 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_283 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_284 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_285 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_286 (0.36, 0.36, 0.36) (0.36, 0.36, 0.36)
crc_287 (0.36, 0.36, 0.36) (0.36, 

In [6]:
image_files
mask_files

['/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_266.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_267.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_268.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_269.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_270.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_271.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_272.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_273.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_274.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_275.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_276.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_277.nii.gz',
 '/home/workspace/research/AMP_mysef_3D_Cls/data/labelsTs/crc_278.nii.gz',
 '/home/workspace/researc